# Using AI and NLP to Improve Your Cooking

Michael Humkey, Conor Marsten

In [104]:
import urllib
import urllib.request

In [105]:
url = "http://food2fork.com/api/search?"
args = {'key' : 'd8b2df92a9cb994f2009b2be8410c1a3', 'q' : 'spaghetti', 'sort' : 'r'}
data = urllib.parse.urlencode(args)
req = urllib.request.Request(url+data, headers={'User-Agent': 'Mozilla/5.0'})
response = urllib.request.urlopen(req).read()

In [106]:
import json

In [107]:
response = json.loads(response)
recipes = response['recipes']

In [108]:
from bs4 import BeautifulSoup
import requests

In [109]:
def getReviews(recipes, i=0):

    recipe = recipes[i]
    url = "http://food2fork.com/api/get?"
    data = urllib.parse.urlencode({'key' : 'd8b2df92a9cb994f2009b2be8410c1a3', 'rId' : recipe['recipe_id']})
    req = urllib.request.Request(url+data, headers={'User-Agent': 'Mozilla/5.0'})
    recipe = json.loads(urllib.request.urlopen(req).read())['recipe']
    bs =  BeautifulSoup(requests.get(recipe['source_url']).content, 'html.parser')
    reviews = []

    if (recipe['publisher'] == '101 Cookbooks'): #1
        reviews = bs.find_all(attrs={'class': 'card-body'})

    elif (recipe['publisher'] == 'BBC Good Food'):
        reviews = bs.find_all(attrs={'class': 'field-item even'})

    elif (recipe['publisher'] == 'Closet Cooking' 
           or recipe['publisher'] == 'Eats Well With Others' 
           or recipe['publisher'] == "Lisa's Kitchen"
           or recipe['publisher'] == 'A Spicy Perspective'
           or recipe['publisher'] == 'Naturally Ella'
           or recipe['publisher'] == 'Pastry Affair'):
        reviews = bs.find_all(attrs={'class': 'comment-body'})

    elif (recipe['publisher'] == 'Food Republic'):
        reviews = bs.find_all(attrs={'class': '_5mdd'}) #uses facebook comments plugin

    elif (recipe['publisher'] == 'PBS Food'):
        reviews = bs.find_all(attrs={'class': 'post-message'})

    elif (recipe['publisher'] == 'Simply Recipes'
          or recipe['publisher'] == 'Homesick Texan'
          or recipe['publisher'] == 'Tasty Kitchen'):
        reviews = bs.find_all(attrs={'class': 'comment-text'})

    elif (recipe['publisher'] == 'Two Peas and Their Pod'
          or recipe['publisher'] == 'Cookie and Kate'
          or recipe['publisher'] == "Elana's Pantry"
          or recipe['publisher'] == 'My Baking Addiction'
          or recipe['publisher'] == 'Smitten Kitchen'
          or recipe['publisher'] == 'Vintage Mixer'
          or recipe['publisher'] == 'Cookin Canuck'
          or recipe['publisher'] == 'Healthy Delicious'
          or recipe['publisher'] == 'Steamy Kitchen'
          or recipe['publisher'] == "What's Gaby Cooking"
          or recipe['publisher'] == 'Bunky Cooks'
          or recipe['publisher'] == 'Serious Eats'):
        reviews = bs.find_all(attrs={'class': 'comment-content'})

    elif (recipe['publisher'] == 'All Recipes'):
        reviews = bs.find_all(itemprop="reviewBody")

    elif (recipe['publisher'] == 'Big Girls Small Kitchen'
          or recipe['publisher'] =='Jamie Oliver'
          or recipe['publisher'] == 'The Pioneer Woman'):
        reviews = bs.find_all(attrs={'data-role': 'message'})

    elif (recipe['publisher'] == 'Framed Cooks'
          or recipe['publisher'] == 'Picky Palate'):
        reviews = bs.find_all(attrs={'class': 'comment even thread-even depth-1'})

    elif (recipe['publisher'] == 'Bon Appetit'):
        reviews = bs.find_all(attrs={'class': 'review-body'})

    elif (recipe['publisher'] == 'Epicurious'):
        reviews = bs.find_all(attrs={'class': 'review-text'})

    elif (recipe['publisher'] == 'Cookstr'):
        reviews = bs.find_all(attrs={'class': 'commentText'})

    elif (recipe['publisher'] == 'Panini Happy'):
        #careful, sloppy html
        reviews = bs.find_all(attrs={'class': 'format_text'})

    elif (recipe['publisher'] == 'Real Simple'):
        reviews = bs.find_all(attrs={'class': 'comment_txt'})
        
    elif (recipe['publisher'] == 'Chow'):
        reviews = bs.find_all(itemprop="comment")

    elif (recipe['publisher'] == 'Delishhh'):
        reviews = bs.find_all(attrs={'class': 'commentmeta'})

    elif (recipe['publisher'] == 'Food Network'):
        reviews = bs.find_all(attrs={'class': 'gig-comment-body'})    
    else:
        print(recipe['publisher'])
        getReviews(recipes, i+1)
        
    return reviews, recipe

In [110]:
reviews, recipe = getReviews(recipes)
revs = [rv.get_text().replace('\n', ' ').replace('\r', '') for rv in reviews]

## Performing sentiment analysis

In [111]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia

In [112]:
analyzer = sia()
scored = []
for rev in revs:
    res = analyzer.polarity_scores(rev)
    tup = (rev, res)
    scored.append(tup)

In [131]:
def getKey(tup):
    return tup[1]['compound']

scored = list(reversed(sorted(scored, key=getKey)))

In [132]:
scored

[('I’m in heaven!   This is my new favorite Gluten Free dish and I’m especially excited for this summer.  Made this as written except used flat leaf parlsey .  I also added a little olive oil and avocado oil in the blender to help it along and it made a really creamy sauce.  Used Gluten free pasta and stored the leftovers in the fridge with the avocado pit and it stayed that beautiful green till the next day.  My husband and I are already planning a super summer dinner of this pasta, lobster tails, a white chilled wine and melon for dessert.  Oh my goodness, I can’t wait. ',
  {'compound': 0.9824, 'neg': 0.0, 'neu': 0.754, 'pos': 0.246}),
 ('Okay, a little late to the game here but I had to post nonetheless.  I’ve had this recipe sitting in my  “to try” pile for ages and never got around to it.  My husband and I were both a little skeptical but we shouldn’t have been – it was great.  I did have to tweak it a little bit based on what we had on hand – no fresh garlic, so I used powder, n

## Checking for foods

In [133]:
from nltk.corpus import wordnet as wn
food = wn.synset('food.n.02')

In [137]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///usda.sql3')
connection = engine.connect()